# Reading data

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import os

In [2]:
import psycopg2
import pandas as pd

conn_string = "host='localhost' dbname='weather_env' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
cur.execute("""SELECT * FROM environment_data.pivoted;""")
records = cur.fetchall()
cur.close()
df = pd.DataFrame(records, columns=[x[0] for x in cur.description])
df.index = pd.to_datetime(df[['measure_year', 'measure_month', 'measure_day']].rename({'measure_year': 'year', 'measure_month': 'month', 'measure_day': 'day'}, axis=1))

# Basic statistics

In [ ]:
ddf = df.describe([.01, .10, .25, .5, .75, .90, .99]).T.sort_values('count')
ddf

# Creating normal plots

In [ ]:

index = pd.DatetimeIndex([datetime(y, m, d) for y, m, d in df[['measure_year', 'measure_month', 'measure_day']].values])

for name in df.columns:
    plt.plot(index, df[name])
    plt.title(name)
    plt.show()

In [ ]:
for name in df.columns[3:]:
    plt.hist(df[name], 100, density=True, histtype='step')
    plt.title(name)
    plt.show()

In [ ]:
df[['arnhem_waterlevel']].groupby([df.index.month, df.index.day]).mean().plot()
df[['arnhem_waterlevel']].groupby(df.index.isocalendar().week).mean().plot()
plt.show()

In [ ]:
for name in df.columns[3:]:
    # not equidistant so incorrect
    plt.magnitude_spectrum(df[name].dropna(), 1/24*3600)
    plt.title(name)
    plt.show()

# Correlation

With np.corrcoef

In [ ]:
#.drop(['measure_year', 'measure_month', 'measure_day'], axis=1)
corr_df = df.corr()
corr_df.loc[corr_df.abs().sort_values('arnhem_waterlevel').index, 'arnhem_waterlevel'].iloc[::-1]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(corr_df.values)
im.set_clim(-1, 1)

ax.xaxis.set(ticks=list(range(len(corr_df.columns))), ticklabels=corr_df.columns)
plt.xticks(rotation=90)
ax.yaxis.set(ticks=list(range(len(corr_df.columns))), ticklabels=corr_df.columns)

cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
plt.show()

# Create correlation Shift

With np.correlate full, doesn't make any sense when the the timestep is unequidistant

In [ ]:
shift_df = pd.DataFrame()
cor_values = df.T  #drop(['measure_year', 'measure_month', 'measure_day'], axis=1).T

for _, d1 in cor_values.iterrows():
    for _, d2 in cor_values.iterrows():
        if d1.name != d2.name:
            print(d1.name, d2.name)
            sub_df = pd.concat([d1, d2], axis=1).dropna()
            subcor = np.correlate(*(sub_df.values.T), 'full')
            
            shift_df.loc[d1.name, d2.name] = np.argmax(subcor) - subcor.shape[0]/2
shift_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(shift_df.values)
im.set_clim(-1, 1)

ax.xaxis.set(ticks=list(range(len(shift_df.columns))), ticklabels=shift_df.columns)
plt.xticks(rotation=90)
ax.yaxis.set(ticks=list(range(len(shift_df.columns))), ticklabels=shift_df.columns)

cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
plt.show()

# Explain np.correlate

np.correlate tells how much `b` is forward of `a`

In [ ]:
x = np.linspace(0, 30, 300)
a = np.sin((x/3))
b = np.cos((x/3)+1.*np.pi)
# b = np.cos((x/3Z)+.1*np.pi)

plt.plot(x, a)
plt.plot(x, b)
plt.title('original comparison')
plt.show()

In [ ]:
cor = np.correlate(a, b, 'full')
xc = np.linspace(0, cor.size, cor.size)
xc -= xc.mean()
xc = xc.round().astype(int)
plt.plot(xc, cor)
plt.title('correlation')
plt.show()
shift = xc[np.argmax(cor)]
shift

In [ ]:
xb = x+shift*np.mean(np.diff(x))

plt.plot(x, a)
plt.plot(xb, b)
plt.title(f'shifted b {shift} samples forward to match')
plt.show()

# Split in features and target

In [9]:
ddf = df.describe().T.sort_values('count')
max_dict = pd.Series()

for n in range(100):
    n = (n+1)/100
    max_dict[n] = np.divide(*df[ddf.loc[ddf['count'] > max(ddf['count'].values)*n].index[::-1]].dropna().shape)

max_dict.loc[max_dict!=22996.0].sort_values(ascending=False).head(20)

C:\Users\woute\AppData\Local\Temp\ipykernel_17232\424169150.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  max_dict = pd.Series()
C:\Users\woute\AppData\Local\Temp\ipykernel_17232\424169150.py:6: RuntimeWarning: divide by zero encountered in divide
  max_dict[n] = np.divide(*df[ddf.loc[ddf['count'] > max(ddf['count'].values)*n].index[::-1]].dropna().shape)


1.00        inf
0.56    11547.6
0.50    11547.6
0.51    11547.6
0.52    11547.6
0.53    11547.6
0.54    11547.6
0.55    11547.6
0.49    11547.6
0.57    11547.6
0.59    11547.6
0.60    11547.6
0.61    11547.6
0.62    11547.6
0.58    11547.6
0.45     6300.0
0.42     6300.0
0.43     6300.0
0.44     6300.0
0.46     6300.0
dtype: float64

In [ ]:
ddf = df.describe().T.sort_values('count')
n = 0.2
subset = ddf.loc[ddf['count'] > max(ddf['count'].values)*n]
subdf = df[subset.index[::-1]].dropna()
desdf = subdf.describe().T
desdf.index = pd.MultiIndex.from_tuples(map(lambda x: x.replace('_ch', '(ch)').replace('_de', '(de)').split('_'), desdf.index.values.tolist()))
desdf.sort_index()

In [ ]:
target = subdf.pop('arnhem_waterlevel')
features_name, target_name = subdf.columns.values, target.name
features, target = subdf.values, target.values
features.shape, target.shape

# Importance model

In [ ]:
from sklearn.linear_model import Ridge, Lasso

model = Ridge().fit(features, target)
importance = np.abs(model.coef_)
importance = model.coef_

In [ ]:
sorting = np.argsort(importance)

plt.barh(width=importance[sorting], y=features_name[sorting])
plt.title("Feature importances via coefficients")
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

threshold = np.sort(importance)[-5] + 0.01

sfm = SelectFromModel(model, threshold=threshold).fit(features, target)
print(f"Features selected by SelectFromModel: {features_name[sfm.get_support()]}")

# Imputation

SimpleImputer is quick but the distribution changes to much

KNNImputer is slow but the results are meh

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer

In [ ]:
df.dropna(subset=['arnhem_waterlevel'], inplace=True)

In [ ]:
target = df.pop('arnhem_waterlevel')

In [ ]:
target_nans = ~target.isna()
features, target = df.loc[target_nans], target.loc[target_nans]

In [ ]:
ddf = features.describe().T.sort_values('count')

print('original', features.shape)
for main_n in range(10):
    subset = ddf.loc[ddf['count'] > np.max(ddf['count'].values) * (main_n/10)]
    subdf = features[subset.index[::-1]]

    for sub_n in range(11):
        ind = (~subdf.isna()).mean(axis=1) >= sub_n/10
        path = f'subsets//{main_n}-{sub_n}.csv'
        subdf.loc[ind].dropna(axis=1, how='all').to_csv(path)

In [ ]:
def knn_impute(formpath, topath):
    print(formpath)
    subset = pd.read_csv(formpath, index_col=0)

    imp_mean = KNNImputer(
        missing_values=np.nan,
        n_neighbors=270,
        keep_empty_features=True
    )

    pd.DataFrame(
        imp_mean.fit_transform(subset.values),
        index=subset.index,
        columns=subset.columns
    ).to_csv(topath)

In [ ]:
import glob

files = glob.glob('subsets\\*.csv')

In [ ]:
# knn_impute('subsets\\8-8.csv', 'imputed_subsets\\8-8.csv')
for file in files[::-1]:
    knn_impute(file, file.replace('subsets', 'imputed_subsets'))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
subplots = []
for name in df.columns[3:]:
    df_count, df_bin = np.histogram(df[name].dropna().values, bins=200, density=True)
    plt.stairs(df_count, df_bin, label='original', linewidth=2)

    subplot = pd.Series(dtype=object, name=name)
    for file in files:
        dv = pd.read_csv(file.replace('subsets', 'imputed_subsets'), index_col=0)

        try:
            dv_count, dv_bin = np.histogram(dv[name].values, bins=200, density=True)

            in_match = pd.concat([
                pd.Series(df_count, index=df_bin[:-1]),
                pd.Series(dv_count, index=dv_bin[:-1])
            ], axis=1)
            in_match = in_match.groupby(in_match.index.astype(int)).mean().dropna()
            
            subplot[np.min(np.corrcoef(*in_match.T.values))] = (file.replace('subsets', '').replace('.csv', ''), dv_count, dv_bin)
        except KeyError:
            pass
    
    subplots.append(pd.Series({v[0]: k for k, v in subplot.items()}, name=name))
    subplot = subplot.sort_index(ascending=False).iloc[:9]
    for _, (k, *v) in subplot.items():
        plt.stairs(*v, label=k, linestyle='--', linewidth=.5)

    
    plt.title(name)
    plt.legend(ncols=10, loc=1)
    plt.show()

In [ ]:
pd.concat(subplots, axis=1).sum(axis=1).sort_values(ascending=False).iloc[:5]

In [ ]:

index = pd.DatetimeIndex([datetime(y, m, d) for y, m, d in df[['measure_year', 'measure_month', 'measure_day']].values])

for name in df.columns:
    plt.plot(index, df[name])
    plt.title(name)
    plt.show()